## Load Data
### Uber Eats Data

In [3]:
from sql_reader import connect_to_sql_server, read_dataframe_from_sql

In [4]:
conn, cursor = connect_to_sql_server()

In [5]:
query = """
SELECT m.id AS item_id, m.menu_name AS item_name, m.menu_category, m.menu_item_description, m.restaurant_name, m.restaurant_id, r.restaurant_type
FROM Menu_mds_sorted m
JOIN Restaurants_mds r ON m.restaurant_id = r.id
WHERE m.restaurant_id <= 10000
"""
df_uber = read_dataframe_from_sql(query, conn)
df_uber.head()

/Users/zhiweizhang/Documents/Canada/UBC/capstone/heymate-report/script/sql_reader.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Read 402324 rows from SQL Server.


,item_id,item_name,menu_category,menu_item_description,restaurant_name,restaurant_id,restaurant_type
0,5-1,Pork Chop Rice with Gravy Plate,Picked for you,None,Nelson Brothers Cafe (17th St N),5,"Breakfast and Brunch, Burgers, Sandwiches"
1,5-2,Full Sausage (2 pcs) with 2 Eggs,Picked for you,2 pieces.,Nelson Brothers Cafe (17th St N),5,"Breakfast and Brunch, Burgers, Sandwiches"
2,5-3,Bacon and Egg with Cheese Breakfast Sandwich,Picked for you,None,Nelson Brothers Cafe (17th St N),5,"Breakfast and Brunch, Burgers, Sandwiches"
3,5-4,Double Cheese Burger,Picked for you,Grilled or fried patty with cheese on a bun.,Nelson Brothers Cafe (17th St N),5,"Breakfast and Brunch, Burgers, Sandwiches"
4,5-5,Full Bacon (3 pcs) with 2 Eggs,Picked for you,3 pieces.,Nelson Brothers Cafe (17th St N),5,"Breakfast and Brunch, Burgers, Sandwiches"


### Heymate Internal Data

In [6]:
query = """
SELECT *
FROM Internal_menu_mds
"""
df_heymate = read_dataframe_from_sql(query, conn)
conn.close()
df_heymate.head()

Read 4866 rows from SQL Server.


/Users/zhiweizhang/Documents/Canada/UBC/capstone/heymate-report/script/sql_reader.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,item_id,row_id,item_name,menu_category,restaurant_name,restaurant_id,restaurant_type
0,459-1,1,Spicy Combo Spicy Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
1,459-2,21,Dynamite Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
2,459-3,32,Kids Roll Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
3,459-4,43,Salmon/Tuna Combo,Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant
4,459-5,54,Chef's Choice Sushi (10pcs),Combos,Ajishou Japanese Cuisine (Brentwood),459,Japanese Restaurant


## LLM Data Cleaning

In [7]:
from openai import OpenAI
from datetime import datetime, timezone
import requests
import time
import json
from typing import Iterable, List, Dict, Any, Union
import pandas as pd

### Configuration

In [8]:
ALLOWED_RESTAURANT_TYPES = [
    "acai shop", "afghani restaurant", "african restaurant", "american restaurant", "asian restaurant",
    "bagel shop", "bakery", "bar", "bar and grill", "barbecue restaurant", "brazilian restaurant",
    "breakfast restaurant", "brunch restaurant", "buffet restaurant", "cafe", "cafeteria", "candy store",
    "cat cafe", "chinese restaurant", "chocolate factory", "chocolate shop", "coffee shop", "confectionery",
    "deli", "dessert restaurant", "dessert shop", "diner", "dog cafe", "donut shop", "fast food restaurant",
    "fine dining restaurant", "food court", "french restaurant", "greek restaurant", "hamburger restaurant",
    "ice cream shop", "indian restaurant", "indonesian restaurant", "italian restaurant", "japanese restaurant",
    "juice shop", "korean restaurant", "lebanese restaurant", "meal delivery", "meal takeaway", "mediterranean restaurant",
    "mexican restaurant", "middle eastern restaurant", "pizza restaurant", "pub", "ramen restaurant", "restaurant",
    "sandwich shop", "seafood restaurant", "spanish restaurant", "steak house", "sushi restaurant", "tea house",
    "thai restaurant", "turkish restaurant", "vegan restaurant", "vegetarian restaurant", "vietnamese restaurant", "wine bar"
]

restaurant_type_list = ", ".join(f'"{r}"' for r in ALLOWED_RESTAURANT_TYPES)

In [9]:
MODEL_NAME = "o4-mini"
SYSTEM_PROMPT = """
You are a Menu Data Extractor.

Input: A list of menu items. Each item includes:
  - restaurant_name
  - restaurant_type
  - item_name
  - menu_item_description (may be empty)
  - menu_category (may be empty)

Output: a JSON array (same order) with:
  - dish_base : string (the primary dish name, e.g., "pizza")
  - dish_flavor : string[] (up to 5 flavour descriptors, e.g., "pepperoni")
  - is_combo : boolean
  - restaurant_type_std : string

Rules:
  • Use lowercase American English, and translate non-English terms into English based on Merriam-Webster and AP Stylebook.
  
  • dish_base:
      - Should be the main identity of the dish (e.g., "pizza", "lo mein", "fried rice").
      - Remove size indicators (e.g., "small", "XL"), quantity counts (e.g., "3 pcs"), and side items.
      - If the base is unclear or ambiguous, use "unknown".
      - Singular form only (e.g., "sandwich" not "sandwiches").
      - Use menu_name and context from other inputs to identify dish_base.

  • dish_flavor:
      - Up to 5 descriptors of flavor, cooking style, toppings, sauces, etc.
      - Each tag must be no more than two words.
      - DO NOT repeat dish_base (e.g., if dish_base is "shrimp fried rice", do not include "fried rice" again) unless meaning is added.
      - Use singular form (e.g., "egg" not "eggs")and lowercase.

  • is_combo:
      - Set to `true` if the item clearly bundles multiple components (e.g., main dish + sides + drink).

  • restaurant_type_std:
      - Must match one of the following values exactly: {restaurant_type_list}
      - Use the exact spelling and spacing and do NOT add or modify words (e.g., don't add "restaurant" if it's not part of the allowed type).
      - Start by checking whether the input field `restaurant_type` contains or closely matches any of the allowed types.
        - If it partially matches or contains keywords, normalize it accordingly (e.g., "fast food, burgers" → "fast food restaurant").
      - If no reliable match is found from `restaurant_type`, use other inputs to infer. 

      
  • Only return raw JSON (no Markdown, no backticks (```), no extra labels). Output must be valid JSON and preserve the original input order.

Example input:
{
    "restaurant_name": "Pizza World",
    "restaurant_type": "Fast Food, Pizza, Burgers",
    "item_name": "Bacon Cheeseburger Combo",
    "menu_item_description": "Cheeseburger with bacon, fries, and a large coke.",
    "menu_category": "Combo"
  },
  {
    "restaurant_name": "Sushi & Wok",
    "restaurant_type": "Japanese, Chinese",
    "item_name": "Shrimp Fried Rice",
    "menu_item_description": "Grilled shrimp with soy glaze, served over fried rice.",
    "menu_category": ""
  }

Expected output:
[
  {
    "dish_base": "cheeseburger",
    "dish_flavor": ["bacon", "fries", "coke"],
    "is_combo": true,
    "restaurant_type_std": "fast food restaurant"
  },
  {
    "dish_base": "fried rice",
    "dish_flavor": ["shrimp", "grilled", "soy glaze"],
    "is_combo": false,
    "restaurant_type_std": "chinese restaurant"
  }
]
"""

In [10]:
class OpenAIConnector:
    def __init__(self, token_path="../credentials/open_ai_token.txt"):
        self.token = self._load_token(token_path)
        self.client = OpenAI(api_key=self.token)
        self.current_prompt = ''

    @staticmethod
    def _load_token(token_path):
        try:
            with open(token_path, "r") as f:
                token = f.read().strip()
                if not token:
                    raise ValueError("Token file is empty.")
                return token
        except FileNotFoundError:
            raise FileNotFoundError(f"Token file not found at {token_path}")
        except Exception as e:
            raise RuntimeError(f"Error reading token: {e}")
            
    # ─────────────────────────────────────────────────────────────
    #  PUBLIC –  main entry point
    # ─────────────────────────────────────────────────────────────
   
    def classify_batch(
        self,
        rows: Union[str, Iterable[Dict[str, str]]],
        model: str = MODEL_NAME,
        timeout_s: int = 60
    ) -> List[Dict[str, Any]]:
        start_time = time.time()
    
        if isinstance(rows, str):
            rows = json.loads(rows)
    
        # Make the message readable, clean JSON string
        user_msg = json.dumps(rows, indent=2)
    
        # Send one request for the entire batch
        resp = self.client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_msg},
            ],
            timeout=timeout_s
        )
    
        # Parse full JSON array
        content = resp.choices[0].message.content
        try:
            results = json.loads(content)
        except json.JSONDecodeError:
            raise ValueError("❌ Failed to parse response as JSON:\n" + content)
    
        elapsed = time.time() - start_time
        print(f"✅ Processed {len(rows)} rows in {elapsed:.2f} seconds.")
        print(f"⏱️ Average time per row: {elapsed / len(rows):.2f} seconds.")
    
        return results

        
    def get_usage_summary(self):
        headers = {
            "Authorization": f"Bearer {self.token}"
        }

        now = datetime.now(timezone.utc)
        start_date = now.replace(day=1).strftime("%Y-%m-%d")
        end_date = now.strftime("%Y-%m-%d")

        # 1. Get usage data
        usage_url = f"https://api.openai.com/v1/dashboard/billing/usage?start_date={start_date}&end_date={end_date}"
        usage_resp = requests.get(usage_url, headers=headers)
        usage_data = usage_resp.json()
        used_usd = usage_data.get("total_usage", 0) / 100.0  # from cents to dollars

        # 2. Get allowance (subscription limit)
        limits_url = "https://api.openai.com/v1/dashboard/billing/subscription"
        limits_resp = requests.get(limits_url, headers=headers)
        limits_data = limits_resp.json()
        hard_limit = limits_data.get("hard_limit_usd", 0)
        soft_limit = limits_data.get("soft_limit_usd", 0)

        remaining = hard_limit - used_usd

        return {
            "used_usd": round(used_usd, 2),
            "soft_limit_usd": round(soft_limit, 2),
            "hard_limit_usd": round(hard_limit, 2),
            "remaining_usd": round(remaining, 2)
        }

### Test on Sample Batch

In [11]:
import random

def qc(df_input, batch_index=None, batch_size=30,  col_mapping=None):
    if col_mapping is None:
        col_mapping = {}

    df = df_input.rename(columns=col_mapping).copy()

    # Fill missing expected cols with empty string
    expected_cols = ["restaurant_name", "restaurant_type", "item_name", "menu_item_description", "menu_category"]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = ""

    if batch_index is None:
        batch_index = random.randint(0, len(df_input) // batch_size) 

    df_batch = df[expected_cols]
    df_batch = df_batch[batch_size * batch_index : batch_size * (batch_index + 1)]

    # Initialize connector and classify
    oai = OpenAIConnector()
    result = oai.classify_batch(df_batch.to_dict(orient='records'))

    # Combine with original
    df_result = pd.DataFrame(result)
    df_combined = pd.concat([df_batch.reset_index(drop=True), df_result], axis=1)

    return df_combined

In [12]:
# Test of Uber Eats data
result_uber = qc(df_uber, batch_index=23, batch_size=20)
result_uber

✅ Processed 20 rows in 32.52 seconds.
⏱️ Average time per row: 1.63 seconds.


,restaurant_name,restaurant_type,item_name,menu_item_description,menu_category,dish_base,dish_flavor,is_combo,restaurant_type_std
0,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Miguel-Angelo,"Buffalo chicken, bacon, and jalapenos. Topped ...",Specialty Pizza Tacos,pizza taco,"[buffalo chicken, bacon, jalapeno, romaine let...",False,italian restaurant
1,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Meatballer,Ruscelli's homemade meatballs and marinara. To...,Specialty Pizza Tacos,pizza taco,"[meatball, marinara, romaine lettuce, tomato, ...",False,italian restaurant
2,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Maui Itali,Ruscelli's Porchetta with jalapenos and pineap...,Specialty Pizza Tacos,pizza taco,"[porchetta, jalapeno, pineapple, alfredo, bals...",False,italian restaurant
3,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Julio Caesar,Diced chicken breast with pepperoni with caesa...,Specialty Pizza Tacos,pizza taco,"[chicken, pepperoni, caesar salad, romaine let...",False,italian restaurant
4,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",The Rancher,"Chicken, bacon, and ranch on a pizza taco",Specialty Pizza Tacos,pizza taco,"[chicken, bacon, ranch]",False,italian restaurant
5,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Pepperoni and Sausage Nachos,None,Pizza Nachos,nacho,"[pepperoni, sausage]",False,italian restaurant
6,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Nachos,"Shredded romaine, tomato, onion, and jalapenos.",Pizza Nachos,nacho,"[romaine lettuce, tomato, onion, jalapeno]",False,italian restaurant
7,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Chicken and Bacon Nachos,None,Pizza Nachos,nacho,"[chicken, bacon]",False,italian restaurant
8,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Meatball Nacho,None,Pizza Nachos,nacho,[meatball],False,italian restaurant
9,Ruscelli's Food Truck at Mojo Pub,"Italian, Exclusive to Eats",Cookie Dough Pizza,crumbled cookie dough baked on pizza dough and...,Dessert,pizza,"[cookie dough, honey bacon]",False,italian restaurant


In [13]:
# Test of Uber Eats data
result_heymate = qc(df_heymate, batch_index=40, batch_size=20)
result_heymate

✅ Processed 20 rows in 24.01 seconds.
⏱️ Average time per row: 1.20 seconds.


,restaurant_name,restaurant_type,item_name,menu_item_description,menu_category,dish_base,dish_flavor,is_combo,restaurant_type_std
0,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Chicken Teri Don,,RICE DON,rice bowl,"[chicken, teriyaki]",False,sushi restaurant
1,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Beef Teri Don,,RICE DON,rice bowl,"[beef, teriyaki]",False,sushi restaurant
2,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Oyako Don,,RICE DON,rice bowl,"[chicken, egg]",False,sushi restaurant
3,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Katsu Don,,RICE DON,rice bowl,[pork cutlet],False,sushi restaurant
4,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Ten Don,,RICE DON,rice bowl,[tempura],False,sushi restaurant
5,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Soy Chicken Karage Don,,RICE DON,rice bowl,"[chicken, karaage, soy]",False,sushi restaurant
6,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Mild Chicken Karage Don,,RICE DON,rice bowl,"[chicken, karaage, mild]",False,sushi restaurant
7,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Spicy Chicken Karage Don,,RICE DON,rice bowl,"[chicken, karaage, spicy]",False,sushi restaurant
8,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Tuna Don,,SUSHI DON,rice bowl,[tuna],False,sushi restaurant
9,Kinkura Sushi + Sake,"Sushi Restaurant, Japanese Restaurant",Salmon Don,,SUSHI DON,rice bowl,[salmon],False,sushi restaurant


### Test on Standardized Restaurant Type

In [14]:
def test_restaurant_type_validity(df, allowed_types, column_name="restaurant_type_std"):
    values = df[column_name].dropna().str.strip().str.lower().unique()
    allowed_set = set([v.strip().lower() for v in allowed_types])
    
    invalid_values = set(values) - allowed_set
    passed = len(invalid_values) == 0

    if passed:
        print("✅ All restaurant_type_std values are valid.")
    else:
        print("❌ Invalid restaurant_type_std values found:")
        for v in invalid_values:
            print("   -", v)
    
    return passed, invalid_values

In [15]:
test_restaurant_type_validity(result_uber, ALLOWED_RESTAURANT_TYPES)
test_restaurant_type_validity(result_heymate, ALLOWED_RESTAURANT_TYPES)

✅ All restaurant_type_std values are valid.
✅ All restaurant_type_std values are valid.


(True, set())